In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
from datetime import timedelta
%matplotlib inline

#from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score

import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

from imblearn.over_sampling import RandomOverSampler


In [2]:
labels = pd.read_csv("../../data/labels_training_set.csv")
df = pd.read_csv("../../data/events_up_to_01062018.csv")

C:\Users\seba\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_persons = pd.read_csv("../../data/trocafone_kaggle_test.csv")

In [4]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["month"] = df["timestamp"].dt.month
df["day"] = df["timestamp"].dt.day

In [5]:
todas_las_personas = df[["person"]].drop_duplicates(subset="person")

In [40]:
compradores = pd.merge(df, labels.loc[labels['label'] == 1], on='person', how='inner' )
compradores_primer_entrada = compradores.groupby('person')['timestamp'].min().reset_index()
compradores_primer_entrada['fecha'] = compradores_primer_entrada['timestamp'].dt.date
compradores_primer_entrada['mes'] = compradores_primer_entrada['timestamp'].dt.month

compradores_primer_entrada['mes'].value_counts()

5    510
4    172
3    152
1     81
2     65
Name: mes, dtype: int64

# Cantidad de veces que realiza un evento dado en el mes 5

In [6]:
eventos_mes_cinco = df.loc[df["month"] == 5]
cant_por_evento_mes_cinco = pd.concat([pd.get_dummies(eventos_mes_cinco['event']),eventos_mes_cinco[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()

cant_por_evento_mes_cinco.columns = ['person', 'ad campaign hit mes 5', 'brand listing mes 5', 'checkout mes 5',\
                                     'conversion mes 5', 'generic listing mes 5', 'lead mes 5', 'search engine hit mes 5',\
                                     'searched products mes 5', 'staticpage mes 5', 'viewed product mes 5', 'visited site mes 5' ]

# Relleno con ceros las personas que no entraron en el mes 5
cant_por_evento_mes_cinco = pd.merge(cant_por_evento_mes_cinco, todas_las_personas, on="person", how='right').fillna(0)

df_train = pd.merge(cant_por_evento_mes_cinco, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product mes 5', 'visited site mes 5', 'staticpage mes 5', 'lead mes 5'])
cant_por_evento_mes_cinco.head()

,person,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,lead mes 5,search engine hit mes 5,searched products mes 5,staticpage mes 5,viewed product mes 5,visited site mes 5
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
2,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
3,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0
4,000e4d9e,19.0,17.0,1.0,0.0,17.0,0.0,5.0,0.0,0.0,339.0,13.0


In [8]:
#veo cuantos users pierdo
cant_por_evento_predecir = pd.merge(cant_por_evento_mes_cinco, df_persons, on="person", how="inner")
cant_por_evento_predecir["person"].count() - df_persons["person"].count()

0

# Distancia entre ultima entrada al sitio y el ultimo dia que tenemos

In [9]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia_de_todos = timestamp_df["timestamp"].max()
ultima_entrada = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
ultima_entrada["dias_hasta_ultimo"] = (ultimo_dia_de_todos - ultima_entrada["ultimo"]).dt.days
ultima_entrada = ultima_entrada[["person", "dias_hasta_ultimo"]]
ultima_entrada.head()

,person,dias_hasta_ultimo
0,0008ed71,14
1,00091926,0
2,00091a7a,66
3,000ba417,5
4,000c79fe,2


In [10]:
# Veo cuantas personas a predecir pierdo
df_predecir = pd.merge(ultima_entrada, df_persons, on="person", how="inner")
df_predecir["person"].count() - df_persons["person"].count()

0

# Cantidad de veces que realiza un evento dado en el mes 4

In [11]:
eventos_mes_cuatro = df.loc[df["month"] == 4]
cant_por_evento_mes_cuatro = pd.concat([pd.get_dummies(eventos_mes_cuatro['event']),eventos_mes_cuatro[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()

cant_por_evento_mes_cuatro.columns = ['person', 'ad campaign hit mes 4', 'brand listing mes 4', 'checkout mes 4',\
                                     'conversion mes 4', 'generic listing mes 4', 'lead mes 4', 'search engine hit mes 4',\
                                     'searched products mes 4', 'staticpage mes 4', 'viewed product mes 4', 'visited site mes 4' ]

# Relleno con ceros las personas que no entraron en el mes 5
cant_por_evento_mes_cuatro = pd.merge(cant_por_evento_mes_cuatro, todas_las_personas, on="person", how='right').fillna(0)

df_train = pd.merge(cant_por_evento_mes_cuatro, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product mes 4', 'visited site mes 4', 'staticpage mes 4', 'lead mes 4'])
cant_por_evento_mes_cuatro.head()

,person,ad campaign hit mes 4,brand listing mes 4,checkout mes 4,conversion mes 4,generic listing mes 4,lead mes 4,search engine hit mes 4,searched products mes 4,staticpage mes 4,viewed product mes 4,visited site mes 4
0,0019e639,1.0,18.0,2.0,0.0,1.0,0.0,1.0,7.0,0.0,38.0,1.0
1,001ca5ee,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,9.0
2,001dfc31,1.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,2.0
3,001e9aea,10.0,4.0,3.0,0.0,0.0,0.0,10.0,0.0,0.0,15.0,4.0
4,002aea56,1.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,12.0,4.0


In [12]:
#veo cuantos users pierdo
predecir = pd.merge(cant_por_evento_mes_cuatro, df_persons, on="person", how="inner")
predecir["person"].count() - df_persons["person"].count()

0

# Distancia en dias de 1er y ultimo dia de entrada al sitio

In [13]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
primer_dia = timestamp_df.groupby("person")["timestamp"].min().rename("primer").reset_index()
distancia_dias = pd.merge(primer_dia, ultimo_dia, on="person", how="inner")
distancia_dias["distan_dias"] = (distancia_dias["ultimo"] - distancia_dias["primer"]).dt.days
distancia_dias = distancia_dias[["person", "distan_dias"]]
distancia_dias.head()

,person,distan_dias
0,0008ed71,0
1,00091926,27
2,00091a7a,0
3,000ba417,9
4,000c79fe,0


In [14]:
# Veo cuantas personas a predecir pierdo
distancia_dias_predecir = pd.merge(distancia_dias, df_persons, on="person", how="inner")
distancia_dias_predecir["person"].count() - df_persons["person"].count()

0

## Cantidad de modelos distintos vistos por cada usuario

In [15]:
modelos_df = df[["person", "model"]]
cant_modelos = modelos_df.groupby("person")["model"].nunique().rename("modelos_dist").reset_index()
cant_modelos.head()

,person,modelos_dist
0,0008ed71,3
1,00091926,36
2,00091a7a,3
3,000ba417,26
4,000c79fe,1


In [16]:
# Veo cuantas personas a predecir pierdo
cant_modelos_predecir = pd.merge(cant_modelos, df_persons, on="person", how="inner")
cant_modelos_predecir["person"].count() - df_persons["person"].count()

0

## Cantidad de días distintos de entrada al sitio

In [17]:
df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()
cant_dias_dist.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,person,cant_dias_dist
0,0008ed71,1
1,00091926,22
2,00091a7a,1
3,000ba417,3
4,000c79fe,1


In [18]:
# Veo cuantas personas a predecir pierdo
cant_dias_dist_predecir = pd.merge(cant_dias_dist, df_persons, on="person", how="inner")
cant_dias_dist_predecir["person"].count() - df_persons["person"].count()

0

# Entro como 'new' en el mes 5

In [23]:
df_mes_5 = df[["person", "month", "new_vs_returning"]].loc[df["month"] == 5]
df_mes_5["nuevo_mes_5"] = (df_mes_5["new_vs_returning"] == "New") * 1
df_mew_5 = df_mes_5[["person", "nuevo_mes_5"]]
df_mew_5 = df_mew_5.sort_values("nuevo_mes_5", ascending=False)
df_mew_5 = df_mew_5.drop_duplicates(subset=['person'], keep='first')
df_mew_5 = pd.merge(df_mew_5, todas_las_personas, on="person", how="right").fillna(0)
df_mew_5.head()

,person,nuevo_mes_5
0,c0f2beb1,1.0
1,9ff9d5d2,1.0
2,ae9e5fa3,1.0
3,bbbe368d,1.0
4,0b7a3edd,1.0


In [24]:
# Veo cuantas personas a predecir pierdo
df_mew_5_predecir = pd.merge(df_mew_5, df_persons, on="person", how="inner")
df_mew_5_predecir["person"].count() - df_persons["person"].count()

0

# Distancia entre ultima entrada al sitio y el ultimo dia del mes 5 que tenemos

In [25]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia_de_todos = timestamp_df["timestamp"].max()
ultima_entrada = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
ultima_entrada["dias_hasta_ultimo"] = (ultimo_dia_de_todos - ultima_entrada["ultimo"]).dt.days
ultima_entrada = ultima_entrada[["person", "dias_hasta_ultimo"]]
ultima_entrada.head()

,person,dias_hasta_ultimo
0,0008ed71,14
1,00091926,0
2,00091a7a,66
3,000ba417,5
4,000c79fe,2


In [26]:
# Veo cuantas personas a predecir pierdo
df_predecir = pd.merge(ultima_entrada, df_persons, on="person", how="inner")
df_predecir["person"].count() - df_persons["person"].count()

0

## Promedio de entradas por dia

In [27]:
entradas_tot = df.groupby("person").agg({"person": "count"})
#cant_entradas_tot = entradas_tot[["person", "entradas_tot"]]
entradas_tot.columns = ["cant_entradas"]
entradas_tot.reset_index(inplace=True)

df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()

entradas_tot_y_dias = pd.merge(entradas_tot, cant_dias_dist, on="person", how="inner")
entradas_tot_y_dias["promedio_por_dia"] = (entradas_tot_y_dias["cant_dias_dist"] / entradas_tot_y_dias["cant_entradas"])
entradas_tot_y_dias.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,person,cant_entradas,cant_dias_dist,promedio_por_dia
0,0008ed71,6,1,0.166667
1,00091926,448,22,0.049107
2,00091a7a,10,1,0.100000
3,000ba417,206,3,0.014563
4,000c79fe,17,1,0.058824


In [28]:
entradas_tot_y_dias = entradas_tot_y_dias[["person", "promedio_por_dia"]]

In [29]:
# Veo cuantas personas a predecir pierdo
entradas_tot_y_dias_predecir = pd.merge(entradas_tot_y_dias, df_persons, on="person", how="inner")
entradas_tot_y_dias_predecir["person"].count() - df_persons["person"].count()

0

## Promedio de entradas por mes

In [30]:
df_mes = df[["person", "month"]]
cant_meses_dist = df_mes.groupby("person")["month"].nunique().rename("cant_meses_dist").reset_index()

entradas_tot_y_meses = pd.merge(entradas_tot, cant_meses_dist, on="person", how="inner")
entradas_tot_y_meses["promedio_por_mes"] = (entradas_tot_y_meses["cant_meses_dist"] / entradas_tot_y_meses["cant_entradas"])
entradas_tot_y_meses = entradas_tot_y_meses[["person", "promedio_por_mes"]]

entradas_tot_y_meses.head()

,person,promedio_por_mes
0,0008ed71,0.166667
1,00091926,0.002232
2,00091a7a,0.100000
3,000ba417,0.004854
4,000c79fe,0.058824


In [31]:
# Veo cuantas personas a predecir pierdo
entradas_tot_y_meses_predecir = pd.merge(entradas_tot_y_meses, df_persons, on="person", how="inner")
entradas_tot_y_meses_predecir["person"].count() - df_persons["person"].count()

0

# Mes que entró por primera vez

In [44]:
primer_entrada = df.groupby('person')['timestamp'].min().reset_index()
primer_entrada['mes_primer_entrada'] = primer_entrada['timestamp'].dt.month
primer_entrada = primer_entrada[['person', 'mes_primer_entrada']]

# Merge de los features

In [46]:
df_final = pd.merge(cant_por_evento_mes_cinco, ultima_entrada, on='person', how='inner')
df_final = pd.merge(df_final, cant_por_evento_mes_cuatro, on='person', how='inner')
df_final = pd.merge(df_final, distancia_dias, on='person', how='inner')
df_final = pd.merge(df_final, df_mew_5, on='person', how='inner')
df_final = pd.merge(df_final, cant_dias_dist, on='person', how='inner')
df_final = pd.merge(df_final, cant_modelos, on='person', how='inner')
df_final = pd.merge(df_final, entradas_tot_y_dias, on='person', how='inner')
df_final = pd.merge(df_final, entradas_tot_y_meses, on='person', how='inner')
df_final = pd.merge(df_final, primer_entrada, on='person', how='inner')
df_final.head()

,person,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,lead mes 5,search engine hit mes 5,searched products mes 5,staticpage mes 5,...,staticpage mes 4,viewed product mes 4,visited site mes 4,distan_dias,nuevo_mes_5,cant_dias_dist,modelos_dist,promedio_por_dia,promedio_por_mes,mes_primer_entrada
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,1.0,1,3,0.166667,0.166667,5
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27,1.0,22,36,0.049107,0.002232,5
2,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,9,1.0,3,26,0.014563,0.004854,5
3,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,...,0.0,0.0,0.0,0,1.0,1,1,0.058824,0.058824,5
4,000e4d9e,19.0,17.0,1.0,0.0,17.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,14,1.0,9,37,0.021898,0.002433,5


In [47]:
# A csv

df_final.to_csv("features_seba.csv", encoding='utf-8', index=False)